# yvBoost & yveCRV Vaults
> "How has the introduction of the yvBOOST Vault impacted usage of the yveCRV Vault?"

- toc:true
- branch: master
- badges: true
- comments: false
- author: Scott Simpson
- categories: [Curve, Yearn]
- hide: true  


In [13]:
#hide
#Imports & settings
!pip install plotly --upgrade
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib inline
#%load_ext google.colab.data_table
%load_ext rpy2.ipython
%R options(tidyverse.quiet = TRUE)
%R options(lubridate.quiet = TRUE)
%R options(jsonlite.quiet = TRUE)
%R suppressMessages(library(tidyverse))
%R suppressMessages(library(lubridate))
%R suppressMessages(library(jsonlite))
%R suppressMessages(options(dplyr.summarise.inform = FALSE))


The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


dplyr.summarise.inform,[RTYPES.LGLSXP]


In [14]:
#hide
%%R
#Grab base query from Flipside
df_yvBoost = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/d87b9578-9bdd-4ed1-8da9-da0156d7575b/data/latest', simplifyDataFrame = TRUE)
df_yveCRV = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/8dc06609-8d0e-4f5d-b754-e61082b336fa/data/latest', simplifyDataFrame = TRUE)


#fix the column names
names(df_yvBoost)<-tolower(names(df_yvBoost))
names(df_yveCRV)<-tolower(names(df_yveCRV))

#Change the date to date format
df_yvBoost$date <- as.Date(parse_datetime(df_yvBoost$date))
df_yveCRV$date <- as.Date(parse_datetime(df_yveCRV$date))

#create a date sequence from min date to max date
full_date_range <- tibble(date = seq(min(df_yvBoost$date), max(df_yvBoost$date), by = "days"))

#join in the df_yvBoost frame
df_yvBoost <- full_date_range %>%
  left_join(df_yvBoost, by=c('date'))

#Get rid of the dodgy ytoken prices
df_yvBoost <- df_yvBoost %>%
  mutate(ytoken_price = if_else(ytoken_price < 1, NA_real_, ytoken_price))

#fill the na prices
df_yvBoost <- df_yvBoost %>%
    fill(vault_name) %>%
    fill(vault_symbol) %>%
    fill(asset_symbol) %>%
    fill(exposure) %>%
    fill(pricing_symbol, .direction = "downup") %>%
    fill(ytoken_price, .direction = "downup") %>%
    fill(token_price, .direction = "downup") %>%
    replace_na(list(mint_amount = 0, burn_amount = 0, net_ytoken_increase = 0))

#yvBoost tokens on issue
df_yvBoost <- df_yvBoost %>%
  arrange(date) %>%
  mutate(yvBoost_equiv_CRV_deposit = net_ytoken_increase * ytoken_price,
         yvBoost_on_issue = cumsum(net_ytoken_increase))

#define ROI as the annualised 7 day vault return - assume 52 weeks/year
#calculate the 7 day lagged value of ytoken_price
df_yvBoost <- df_yvBoost %>%  
  mutate(ytoken_lag7 = lag(ytoken_price, n=7, order_by = date))  %>%
  mutate(yvBoost_7day_ROI = (ytoken_price - ytoken_lag7) * 52 * 100) %>%
  fill(yvBoost_7day_ROI, .direction = "downup")


#create a date sequence from min date to max date
full_date_range <- tibble(date = seq(min(df_yveCRV$date), max(df_yveCRV$date), by = "days"))

#join in the df_yveCRV frame
df_yveCRV <- full_date_range %>%
  left_join(df_yveCRV, by=c('date'))

#fill the na prices
df_yveCRV <- df_yveCRV %>%
    replace_na(list(threecrv_deposit = 0, yvecrv_minted = 0))

#df_yveCRV tokens on issue
df_yveCRV <- df_yveCRV %>%
  arrange(date) %>%
  mutate(yveCRV_on_issue = cumsum(yvecrv_minted))

#Grab yveCRV prices from coingecko
cg_api = paste('https://api.coingecko.com/api/v3/coins/vecrv-dao-yvault/market_chart/range?vs_currency=usd&from=',
               as.numeric(as.POSIXct(min(df_yveCRV$date))),
               '&to=',
               as.numeric(as.POSIXct(max(df_yveCRV$date))),
               sep = "")


yveCRV_prices = fromJSON(cg_api, simplifyDataFrame = TRUE, flatten = TRUE)

yveCRV_prices <- as_tibble(yveCRV_prices$prices) %>%
  mutate(date = as.Date(as.POSIXct(V1/1000, origin="1970-01-01")),
         yveCRV_price = V2) %>%
  select(date, yveCRV_price)

#join back into the main df
df_yveCRV <- df_yveCRV %>%
  left_join(yveCRV_prices, by = c('date'))

#Grab CRV prices from coingecko
cg_api = paste('https://api.coingecko.com/api/v3/coins/curve-dao-token/market_chart/range?vs_currency=usd&from=',
               as.numeric(as.POSIXct(min(df_yveCRV$date))),
               '&to=',
               as.numeric(as.POSIXct(max(df_yveCRV$date))),
               sep = "")


CRV_prices = fromJSON(cg_api, simplifyDataFrame = TRUE, flatten = TRUE)

CRV_prices <- as_tibble(CRV_prices$prices) %>%
  mutate(date = as.Date(as.POSIXct(V1/1000, origin="1970-01-01")),
         CRV_price = V2) %>%
  select(date, CRV_price)


#define ROI as the annualised 7 day vault return - assume 52 weeks/year
#ROI is 3CRV received over tokens on issue
df_yveCRV <- df_yveCRV %>%  
#  drop_na(price) %>%
  mutate(yveCRV_7day_ROI = if_else(threecrv_deposit == 0, NA_real_, threecrv_deposit / (yveCRV_on_issue*yveCRV_price)) * 52 * 100)  %>%
  fill(yveCRV_7day_ROI, .direction = "down")

rm(list = c('full_date_range','yveCRV_prices', 'cg_api'))

#Want date, yveCRV minted, net yvBoost minted
df <- df_yveCRV %>%
  left_join(df_yvBoost %>% select(date, net_ytoken_increase, yvBoost_on_issue, yvBoost_7day_ROI, yvBoost_equiv_CRV_deposit), by = c('date'))

#fix dates back up
df$date <- as_datetime(df$date)

#create a week field
df <- df %>%
  mutate(week = floor_date(date, 'week'))


#join CRV prices back into the main df
df <- df %>%
  left_join(CRV_prices, by = c('date'))

#roll up by week
df_week <- df %>%
  group_by(week) %>%
  summarise(yveCRV_minted = sum(yvecrv_minted),
            yveCRV_7day_ROI = mean(yveCRV_7day_ROI, na.rm = TRUE),
            yvBoost_minted = sum(net_ytoken_increase),
            yvBoost_7day_ROI = mean(yvBoost_7day_ROI, na.rm = TRUE),
            yvBoost_minted_usd = sum(yvBoost_equiv_CRV_deposit * CRV_price),
            yvBoost_equiv_CRV_deposit = sum(yvBoost_equiv_CRV_deposit),
            yveCRV_minted_usd = sum(yvecrv_minted * CRV_price),
            CRV_price = mean(CRV_price),
            yveCRV_price = mean(yveCRV_price)
            ) %>%
  ungroup() %>%
  arrange(week) %>%
  replace_na(list(yvBoost_minted = 0, yvBoost_equiv_CRV_deposit =0, yvBoost_minted_usd = 0)) %>%
  mutate(yveCRV_on_issue = cumsum(yveCRV_minted),
         yvBoost_on_issue = cumsum(yvBoost_minted),
         yvBoost_equiv_CRV_deposit_on_issue = cumsum(yvBoost_equiv_CRV_deposit),
         yveCRV_on_issue_usd = yveCRV_on_issue * CRV_price,
         yvBoost_on_issue_usd = yvBoost_on_issue * yveCRV_price
  )

# Yearn yVaults, yveCRV-DAO and yvBOOST

## Yearn yVaults
Yearn is a DeFi protocol which automates yield farming.  Users have tokens which they want to hold - Yearn puts those tokens to work by finding the best yield farming opportunities across DeFi.  It does this in a gas efficient way for the user, so even small deposits can get decent returns over time.

Yearn users deposit their tokens into yVaults, and receive a token in return which is proportional to their share of the vault capital.  A yVault is a smart contract with one ore more Strategies sitting behind it.  The Strategies are the yield-farming recipes which are created by clever humans (Strategists) and monitored & managed by bots (Keepers).  The yVault contains logic which automatically allocates the vault deposits to whichever combination of Strategies gives the best return for the users.  The rewards from the yield farming accrue into the vault, so the value of the vault token is always increasing.  When a user withdraws from the yVault, they get more tokens than they deposited.  This additional amount is the yield the vault has earned on their behalf.

Each yVault is structured around a particular underlying token - there are vaults for Eth, USDC, WBTC and many others.  Users can deposit in the yVault native token, or they can deposit using any other token & take advantage of Zaps.  Zaps are smart contracts which take an input token and swap it for the underlying token in a gas efficient manner.  The swap may occur via a number of dexs or dex aggregators, but this is abstracted away for the user.  There is a similar feature when withdrawing - the user can withdraw the underlying token from a yVault, or choose to receive their funds in ETH, WBTC, DAI, USDC or USDT.  It's important to know that whatever token the user deposits  or withdraws, they maintain price exposure to the *underlying token* of the vault whilst deposited. 


## yveCRV yVault
Now the veCRV-DAO yVault (also known as the yveCRV Vault) is a little different to the others.  It starts with the CRV token, the governance & reward token from [Curve](https://curve.fi).  Curve is a dex which specialises in stableswaps - swaps between tokens which have approximately the same value.  Examples are ETH/stETH or swaps between dollar pegged stablecoins.  Curve has optimised their swap code to make these swaps efficient from both a liquidity impact and fee perspected - see this [post](https://scottincrypto.github.io/analytics/curve/2021/09/19/Curve_Stableswaps.html) for a further exploration of Curve & stablecoin swaps.

The CRV token has voting rights in the Curve DAO which makes decisions on the Curve protocol - things like fees, LP rewards, swap parameters and pools launched.  In some Curve pools, liquidity providers receive CRV tokens to incentivise liquidity in the pools.  CRV tokens are also available on the open market.  To encourage users to stay as CRV hodlers, there is a facility to lock CRV tokens into the CRV DAO for a fixed period of up to 4 years.  Users receive veCRV tokens (voting escrow Curve Tokens) for doing this, and more tokens are received the longer the locking period.  veCRV holders can still participate in governance voting, they receive 50% of Curve trading fees and they qualify for boosted rewards (up to 2.5x) when they provide liquidity in Curve.

The fees generated for veCRV holders are collected in the form of 3CRV tokens (shares in the [Curve tripool](https://curve.fi/3pool)), which can be redeemed for stablecoins if desired.  Fee distribution for veCRV holders happens weekly and users need to collect these manually and pay the gas cost for the transactions.

In true Yearn fashion there is a vault & a strategy to maximise the returns from this CRV locking process.  This is the yveCRV yVault and it's different to the other vaults in that you *can't withdraw your tokens*.  Yearn takes CRV tokens and locks them with the CRV DAO for the maximum 4 year period and continually renews this lock.  This maximises the veCRV returns to the yVault.  In addition, all Yearn vaults send 10% of earned CRV into this vault for additional boost.  The returns to the users are in the form of the 3CRV tokens earned by the veCRV - like the CRV staking contract, these are collectable weekly as an income stream, and must be collected manually.  yVault depositors receive yveCRV-DAO tokens as their share in the vault.

## yvBoost yVault

Finally we get to the yvBoost yVault.  This vault builds on the yveCRV yVault, but automates the process of collecting the weekly rewards.  The Strategy behind this vault collects the 3CRV rewards each week, swaps them for more yveCRV then deposits them back into the vault.  The yvBoost vault is a standard Yearn yVault - you can withdraw part or all of your outstanding deposit and any accrued gains at any time.  Working in the native tokens of the yVault, users can deposit or withdraw yveCRV-DAO tokens.  Alternatively, users can take advantage of the Zap function and deposit pretty much any ERC-20 token into the vault.  They can also withdraw using Zap and collect WETH, WBTC, DAI, USDT or USDC.  As the Zap conversions occur on the way in and out, the user maintains price exposure to the yveCRV-DAO tokens whilst deposited in the vault.  

The yveCRV yVault launched 4-5 months before the yvBoost yVault.  We will examine what impact the launch of the yvBoost yVault had on the usage of the yveCRV yVault.

# yVault Usage - Total Value Locked in USD

If we are to understand the impact of the yvBoost vault on the usage of the yveCRV vault, first we must define what we mean by usage.  For a Yearn vault, this is best defined by the Total Value Locked (TVL).  This is the sum of the value of the tokens deposited and withdrawn from the vault by users.  For the yveCRV vault this is simple - as there are no withdrawals, we just need to sum the deposits.  For the yvBoost vault, we need to add the deposits & subtract the withdrawal transactions.  We need a common baseline for comparing TVLs, as:
- yveCRV deposits are denominated in CRV
- yvBoost deposits are denominated in yveCRV-DAO

Let's have a look at the TVL denominated in USD over time for these two vaults, shown in the graph below.  There isn't too much to discern from this - the value of both vaults declined post May 2021.  For the yvBoost vault, this may have been due to withdrawals being higher than deposits, or it could be due to the USD value of yveCRV-DAO tokens dropping.  Same with the yveCRV graph - we know that there are no withdrawals, but were the rises due to price or deposits?  It seems we need an alternative approach.

In [15]:
#@title
#hide_input
#time plot by week for inputs
df_p = %R df_week %>%  select(week, yveCRV_on_issue_usd, yvBoost_on_issue_usd) %>% rename("yveCRV-DAO" = yveCRV_on_issue_usd, "yvBoost" = yvBoost_on_issue_usd) %>% pivot_longer(!week, names_to='measure', values_to='tokens')
fig = px.line(df_p
             , x = "week"
             , y = "tokens"
             , color = 'measure'
             , labels=dict(week="Week", measure="Vault", tokens='USD Value')
             , title= "Vault TVL in USD"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="right",
    x=0.99,
    title_text=None
))
fig.update_yaxes(title_text='Amount (USD)')
fig.update_xaxes(title_text=None)
fig.show()

# yVault Usage - Total Value Locked in CRV Tokens

When users deposit CRV into the yveCRV vault, they receive 1 yveCRV-DAO token for each CRV deposited.  Given that yveCRV-DAO tokens are then deposited into the yvBoost vault, it makes sense to look at the TVL of these two vaults denominated in CRV.  The graph below shows the TVL of the two vaults in equivalent CRV tokens.  Now we can see the underlying usage of the vaults independent of the USD price volatility.  We saw rapid growth of the yveCRV vault to March 2021, then it levelled off considerably.  The launch of yvBoost in April saw rapid takeup of this vault, with the TVL approaching that of the yveCRV vault within 6-7 weeks.  At the same time, the yveCRV vault TVL also rose rapidly.  This growth in yveCRV was driven by the yvBoost growth because yvBoost requires yveCRV-DAO tokens to deposit.  These are obtained either by depositing CRV into yveCRV and minting new tokens, or by purchasing them on the secondary market.  The yvBoost vault makes the choice based on what is best value at the time - we will examine the relative pricing a bit later.


In [16]:
#@title
#hide_input
#time plot by week for inputs
df_p = %R df_week %>%  select(week, yveCRV_on_issue, yvBoost_equiv_CRV_deposit_on_issue) %>% rename("yveCRV-DAO" = yveCRV_on_issue, "yvBoost" = yvBoost_equiv_CRV_deposit_on_issue) %>% pivot_longer(!week, names_to='measure', values_to='tokens')
fig = px.line(df_p
             , x = "week"
             , y = "tokens"
             , color = 'measure'
             , labels=dict(week="Week", measure="Vault", tokens='Equivalent CRV')
             , title= "Vault Equivalent Locked CRV Tokens"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(legend=dict(
    yanchor="bottom",
    y=0.01,
    xanchor="right",
    x=0.99,
    title_text=None
))
fig.update_yaxes(title_text='Equivalent CRV Tokens')
fig.update_xaxes(title_text=None)
fig.show()

The graph below shows the same data as above, but looks at the net token growth over time rather than the TVL of each vault.  Here we see more clearly the impact of yvBoost on the growth of yveCRV.  yveCRV grew steadily from Jan-Mar 2021, then growth dropped to almost zero for a month or so.  The growth kicked off again once yvBoost launched, and slowed in line with the slowing in yvBoost growth at the end of June.

In [17]:
#@title
#hide_input
df_p = %R df_week %>%  select(week, yveCRV_minted, yvBoost_equiv_CRV_deposit) #%>%  pivot_longer(!week, names_to='measure', values_to='tokens')
fig = make_subplots(rows=2, cols=1, subplot_titles=("yveCRV-DAO Net Token Growth", "yvBoost Net Token Growth"))
fig.append_trace(go.Bar(x=df_p["week"], y=df_p["yveCRV_minted"], name="CRV"), row=1, col=1)
fig.append_trace(go.Bar(x=df_p["week"], y=df_p["yvBoost_equiv_CRV_deposit"], name="CRV"), row=2, col=1)
fig.update_layout(width=800, height=600)
fig.update_layout(template="simple_white", showlegend=False)
fig.update_yaxes(title_text='Value in CRV', row=1, col=1)#, range=[0, 3.5e6])
fig.update_yaxes(title_text='Value in CRV', row=2, col=1)#, range=[0, 3.5e6])
fig.show()

# Impact of Pricing

We saw above that yveCRV-DAO tokens were created in the yvCRV vault for use in yvBoost in the period from April-June 2021.  The chart below looks at the relative price of yveCRV tokens to CRV tokens.  Remember that you can have a one-way transaction and convert CRV to yveCRV. In doing so you give up the ability to get your CRV back, but gain a perpetual share of the revenue of the vault in return.  The relative price chart below shows what market participants are valuing this loss of flexibility and future income stream at.  In the period from Feb-June, yveCRV traded at an average of 90% of CRV, roughly a 10% discount.  This dropped to 75% from June to August, then fell dramatically to below 40% in September.  This means that the market is valuing the flexibility of the unencumbered CRV tokens over the income stream by a factor of nearly 3!  With yveCRV trading at this much of a discount, it is not surprising that there had been very low growth in the yveCRV since July 2020.  Any required yveCRV tokens (for depositing in yvBoost) can be purchased on the open market at a steep discount to the underlying CRV tokens.

It's unclear what is driving this pricing mismatch - perhaps the competition for CRV token locking with [Convex](https://www.convexfinance.com/) is causing people to exit their yveCRV positions in search of better yields elsewhere.

In [18]:
#@title
#hide_input
#time plot by week for inputs
df_p = %R df_week %>%  select(week, yveCRV_price, CRV_price)  %>% mutate(ratio = yveCRV_price / CRV_price) #%>% rename("yveCRV-DAO" = yveCRV_price, "CRV" = CRV_price)
fig = px.line(df_p
             , x = "week"
             , y = "ratio"
 #            , color = 'measure'
             , labels=dict(week="Week", ratio="Ratio")
             , title= "yveCRV:CRV Price Ratio"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99,
    title_text=None
))
fig.update_yaxes(title_text='yveCRV Price in CRV')
fig.update_xaxes(title_text=None)
fig.show()

# Vault Return on Investment

The pricing above is curious - why is yveCRV valued at such a steep discount to CRV?  We will see if there is any impact from the Return on Investment of the vaults in question.  

Vault ROI for a normal Yearn yVault is calculated from the price of the yToken relative to the underlying deposited token.  Each Vault has an exchange rate built into it - deposit 1 token, get a bit less than one yvtoken in return.  Over time, 1 deposit token will buy less and less yvtokens - the price is always rising.  For a given investment, the ROI is determined by the following formula, where the Buy & Sell prices are the number of native tokens required to buy the yvtokens:

![ROI Formula](https://github.com/scottincrypto/analytics/raw/master/data/yearn_vault_ROI_calc.gif)

The approach used by the Yearn team on the https://yearn.finance page is to annualise this ROI based on a 7 day rolling period.  We have applied this approach to the yvBoost vault.

The ROI calculations for the yveCRV vault is a little simpler - each week there is a deposit of 3CRV tokens into the vault which are the income stream generated from the locked CRV tokens.  We take the value of the deposited 3CRV tokens over the value of the CRV tokens in the vault over each 7 day period and annualise it to get the vault ROI.  Note, this assumes that users withdraw their returns from the vault.

The ROI of the two vaults are shown in the chart below.  We can see that the yvBoost vault tracks the return of the yveCRV vault, with a slight discount on average.  It's not clear whether there is any impact on yveCRV/yvBoost usage due to ROI as they track each other fairly reliably.  What we do see, however, is an impact on ROI of the yveCRV:CRV pricing ratio above.  We can see the yvBoost ROI flippen the yveCRV ROI in late August - this is a direct result of the steep discount of yveCRV relative to CRV.  The underlying income of the yvBoost vault is driven by the number of CRV tokens locked, but the value of the vault has fallen due to the drop in yveCRV price relative to CRV.  This increases the vault ROI dramatically, amplified by the fact that the underlying 3CRV earnings of the vault are swapped into more yveCRV at a discount. This apparent pricing mismatch is an excellent opportunity to get some great returns from the yvBoost vault.

In [19]:
#@title
#hide_input
#time plot by week for inputs
df_p = %R df_week %>%  select(week, yveCRV_7day_ROI, yvBoost_7day_ROI) %>% rename("yveCRV-DAO" = yveCRV_7day_ROI, "yvBoost" = yvBoost_7day_ROI) %>% pivot_longer(!week, names_to='measure', values_to='tokens')
fig = px.line(df_p
             , x = "week"
             , y = "tokens"
             , color = 'measure'
             , labels=dict(week="Week", measure="Vault", tokens='Equivalent CRV')
             , title= "Vault Annualised 7 Day ROI"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99,
    title_text=None
))
fig.update_yaxes(title_text='Annualised ROI %')
fig.update_xaxes(title_text=None)
fig.show()

# Conclusions

We attempted to answer the question of whether there was an impact on the usage of the yveCRV vault from the introduction of the yvBoost vault - a more improved & automated version of yveCRV.  We saw that the usage of yveCRV had already dropped prior to the introduction of yvBoost, so there was no opportunity for yveCRV usage to fall much further.  The launch of yvBoost, however, generated an upswing of deposits to the yveCRV vault, as yveCRV-DAO tokens were required to deposit into the yvBoost vault.  The relative pricing of yveCRV-DAO to CRV meant that minting fresh yveCRV-DAO tokens was the best way to enter yvBoost.  Since the initial 6 week upswing of yvBoost, we have seen the growth of both vaults level out with yvBoost declining a little.  A surprising observation was the steep drop in value of yveCRV-DAO relative to CRV - it's clear that people want out of their locked CRV positions and are prepared to take a big haircut to do so.

- All on-chain data sourced from the curated tables at [Flipside Crypto](https://flipsidecrypto.com)
- Pricing data sourced from [Coingecko](https://coingecko.com)